In [1]:
!pip install agentpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.2 MB/s eta 0:00:00


In [2]:
import agentpy as ap
import numpy as np
import random, json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns, IPython
from matplotlib import pyplot as plt, cm

In [3]:
'''
Grid cell types

gr grass
us street heading up
ds street heading down
rs street heading east
ls street heading left
ic intersection
cw crosswalk
sw sidewalk
pw able cross walk
ho house
de destination
dw driveway
ob obstacle
'''
gr = -100;
cw, ic, us, ls, ds, rs, sw, pw, ho, de, dw, ob = -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 # Values are clockwise
traffic_light_cooldown, traffic_light, car = 99, 100, 101

In [4]:
class CarAgent(ap.Agent):

    """
    Function to setup the car agents
    """
    def setup(self):
      self.env = self.model.env
      self.random = self.model.random
      self.agentType = "car"

      # Store a list of remaining movements to leave an intersection (queue)
      self.intersection_moves = []

      # 0: up, 1: left, 2: down, 3: right
      # This dictionary maps which directions are available to cars traveling in a certain direction
      self.directions = {
          rs: [3, 3, 3, 3, 3, 0, 2],
          us: [0, 0, 0, 0, 0, 1, 3],
          ds: [2, 2, 2, 2, 2, 1, 3],
          ls: [1, 1, 1, 1, 1, 0, 2]
      }

      self.direction = -1

    """
    Function to choose a new direction to go after the next intersection
    """
    def choose_direction(self):
      tile = self.env.get_tile(self.get_position())
      possibleDirections = self.directions[tile]
      self.direction = self.random.choice(possibleDirections)

    """
    Function to run at each step of the simulation
    """
    def execute(self):

      # Check if the car is allowed to move to the next position

      if len(self.intersection_moves) > 0:
        pos = self.get_position()
        move = self.intersection_moves[0]
        next_position = (pos[0] + move[0], pos[1] + move[1])
        # Check there's no car or pedestrian in the next tile, else, don't move and wait for the file to be free
        if not self.model.has_car(next_position) and not self.model.crosswalk_has_pedestrians(next_position):
          self.env.move_to(self, next_position)
          self.intersection_moves.pop(0)
          # If out of the intersection, choose a new random direction
          if len(self.intersection_moves) == 0:
            self.choose_direction()
      else:
        # Advance according to road direction if there is no obstacle (car, pedestrian or crosswalk)
        next_position = self.get_next_position()
        if next_position is not None and not self.model.has_car(next_position) and not self.model.has_pedestrian(next_position):
          if self.env.get_tile(next_position) != cw and self.env.get_tile(next_position) != pw:
            self.env.move_to(self, next_position)

    """
    Function to get the next position for the car depending on the road it's on
    """
    def get_next_position(self):
      position = self.get_position()

      if len(self.intersection_moves) > 0:
        move = self.intersection_moves.pop(0)
        return (position[0] + move[0], position[1] + move[1])

      # Get direction of street and calculate new position
      tile = self.env.get_tile(position)
      if tile == rs:
        return (position[0], position[1]+1)
      elif tile == ls:
        return (position[0], position[1]-1)
      elif tile == us:
        return (position[0]-1, position[1])
      elif tile == ds:
        return (position[0]+1, position[1])
      else:
        return None

    """
    Function to queue the necessary moves to get the car into the intersection it's waiting at
    """
    def enter_intersection(self):
      enter_direction_tile = self.env.get_tile(self.get_position())
      enter_direction = self.direction_from_tile(enter_direction_tile)
      forward = None
      right = None
      left = None

      # Determine the relative direction movements depending on the facing direction
      # 0: up, 1: left, 2: down, 3: right
      if enter_direction == 0: # up
        forward = (-1, 0)
        right = (0, 1)
        left = (0, -1)
      elif enter_direction == 1: # left
        forward = (0, -1)
        right = (-1, 0)
        left = (1, 0)
      elif enter_direction == 2: # down
        forward = (1, 0)
        left = (0, 1)
        right = (0, -1)
      else: # right
        forward = (0, 1)
        right = (1, 0)
        left = (-1, 0)

      # Determine maneuver (forward, left, right) based on current direction and intended direction
      intersection_path = None
      if enter_direction == self.direction:
        intersection_path = "forward"
      elif (enter_direction + 1)%4 == self.direction:
        intersection_path = "left"
      elif (enter_direction - 1)%4 == self.direction:
        intersection_path = "right"

      # Setup the movement queues based on the maneuver and relative directions
      if intersection_path == "forward":
        self.intersection_moves = [forward, forward, forward, forward, forward]
      elif intersection_path == "left":
        self.intersection_moves = [forward, forward, forward, left, left, left]
      elif intersection_path == "right":
        self.intersection_moves = [forward, forward, right, right]

    """
    Function to change a street tile type to a direction index
    """
    def direction_from_tile(self, tile):
      if tile == rs:
        return 3
      elif tile == ls:
        return 1
      elif tile == us:
        return 0
      elif tile == ds:
        return 2

    """
    Function to get the current position in the grid of the car
    """
    def get_position(self):
      return self.env.positions[self]


In [5]:
class TrafficLightAgent(ap.Agent):
    """
    Setup traffic light agent variables
    """
    def setup(self):
        # Actions are linked to a movement in the grid.
        self.actions = {'up': (-1,0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}
        # Positions nedded to move according to starting location in intersection (top left corner)
        self.differencial = {'up': (1,1), 'down': (0, 0), 'left': (0, 1), 'right': (1, 0)}
        # Traffic light rotation order (currently clockwise)
        self.next_positions = {'up':'left', 'left':'down', 'down':'right', 'right':'up'}

        self.env = self.model.env
        self.p = self.model.p

        self.random = self.model.random
        self.agentType = "light"

        # Positions of cars and crosswalks based on starting position (top left corner)
        self.exit_directions = []
        self.cross_walks = {}
        self.top_left_pos = None

        # Traffic light variables
        self.counter = self.p.traffic_light_step_duration # Time the light has spent in green or yellow (passing or cooldown mode)
        self.intersection_position = self.random.choice(['up', 'down', 'left', 'right']) # which direction of the street the traffic light is letting go through FIXME: traffic light may not start in the upper left cell
        self.cooldown = False # Whether the traffic light is on cooldown


    """
    Function to calculate the exits and crosswalks of each side of the intersection
    """
    def find_exits(self):
        # Starting location (top left corner of intersection)
        self.top_left_pos = self.env.positions[self]
        # Move to actual starting position
        self.env.move_by(self, self.differencial[self.intersection_position])

        # Locations of cars that need to pass according to intersection position
        left_exit = (self.top_left_pos[0], self.top_left_pos[1]-2)
        right_exit = (self.top_left_pos[0]+1, self.top_left_pos[1]+3)
        down_exit = (self.top_left_pos[0]+3, self.top_left_pos[1])
        up_exit = (self.top_left_pos[0]-2, self.top_left_pos[1]+1)

        self.exit_directions = [up_exit, left_exit, down_exit, right_exit]

        # Location of crosswalks that need to pass according to intersection position
        left_cross_walk = [(self.top_left_pos[0], self.top_left_pos[1]-1), (self.top_left_pos[0]+1, self.top_left_pos[1]-1)]
        right_exit = [(self.top_left_pos[0], self.top_left_pos[1]+2), (self.top_left_pos[0]+1, self.top_left_pos[1]+2)]
        down_exit = [(self.top_left_pos[0]+2, self.top_left_pos[1]), (self.top_left_pos[0]+2, self.top_left_pos[1]+1)]
        up_exit = [(self.top_left_pos[0]-1, self.top_left_pos[1]), (self.top_left_pos[0]-1, self.top_left_pos[1]+1)]

        self.cross_walks = {'down': left_cross_walk, 'left': up_exit, 'up': right_exit, 'right': down_exit}

        # Enable the corresponding crosswalk
        self.allow_cross_walk(self.intersection_position)


    """
    Function for each step of the simulation
    """
    def execute(self):
        # Traffic light time counter (Green mode)
        if self.counter <= 0 and not self.cooldown:
          self.counter = self.p.traffic_light_step_cooldown
          self.cooldown = True

        # Traffic light time counter (Yellow mode)
        if self.counter <= 0 and self.cooldown:
          self.start_next_cycle()
          self.cooldown = False

        # If light is in green (pass mode)
        if not self.cooldown:
          # Check if a car has a green light and is ready to move
          next_car_position = self.next_car_position()
          if self.is_car_in_position(next_car_position):
            # Get agent in 'position next_car_position'
            carAgent = list(self.env.grid[next_car_position][0])[0]
            carAgent.enter_intersection()

        self.counter -= 1

        self.record('status', not self.cooldown)

    """
    Get the position of the car it should be letting through
    """
    def next_car_position(self):
        next_car_orientation = self.actions[self.intersection_position]
        dx, dy = -2 * next_car_orientation[0], -2 * next_car_orientation[1]

        x, y = self.get_position()
        return x + dx, y + dy

    """
    Get position of agent in environment
    """
    def is_car_in_position(self, pos):
        N, M = self.model.p.street.shape
        if pos in self.env.positions.values():
          return True

    """
    Get position of agent in environment
    """
    def get_position(self):
        return self.env.positions[self]

    """
    Move agent to next position and reset counter
    """
    def start_next_cycle(self):
        # Close down the intersection
        self.disallow_cross_walk(self.intersection_position)

        # Order (Clockwise)
        self.intersection_position = self.next_positions[self.intersection_position]

        # Random (Choose randomly which lane to allow next)
        # self.intersection_position = self.random.choice(['up', 'down', 'left', 'right'])

        # Calculate new position
        new_pos = (self.top_left_pos[0] + self.differencial[self.intersection_position][0],
                   self.top_left_pos[1] + self.differencial[self.intersection_position][1])

        # Move traffic light to new lane and allow new crosswalk
        self.env.move_to(self, new_pos)
        self.allow_cross_walk(self.intersection_position)

        # Restart counter
        self.counter = self.p.traffic_light_step_duration

    """
    Get opposite direction
    """
    def get_opposite_direction(self, dir):
      if dir == "down": return "up"
      elif dir == "up": return "down"
      elif dir == "left": return "right"
      elif dir == "right": return "left"

    """
    Allow both paralel crosswalks to allowed lane
    """
    def allow_cross_walk(self, dir):
      for tile in self.cross_walks[dir]:
        self.env.change_tile(tile, pw)
      for tile in self.cross_walks[self.get_opposite_direction(dir)]:
        self.env.change_tile(tile, pw)

    """
    Disallow both paralel crosswalks to allowed lane
    """
    def disallow_cross_walk(self, dir):
      for tile in self.cross_walks[dir]:
        self.env.change_tile(tile, cw)
      for tile in self.cross_walks[self.get_opposite_direction(dir)]:
        self.env.change_tile(tile, cw)

In [6]:
class PedestrianAgent(ap.Agent):

    """
    Setup pedestrian agent variables
    """
    def setup(self):
      self.env = self.model.env
      # Blackboard forsharing update Q function with other agents
      self.blackboard = self.model.blackboard
      self.actions = self.blackboard.actions
      self.random = self.model.random

      # Avoid putting pedestrians over each other
      self.display_offset = (self.random.uniform(-0.2, 0.2), self.random.uniform(-0.2, 0.2))

      # Choose a random building to go to (destination)
      self.destination = self.random.choice(self.model.destinations)

      self.agentType = "pedestrian"

      # Pedestrian variables
      self.inside_intersection = False
      self.training = False
      self.reward = 0

      # Statistics variables
      self.finish_time = -1
      self.wait_time = 0
      self.distance_moved = 0

    """
    Q-Learning.
    Q update dict is shared with other agents for optimized training
    """
    def train(self, episodes):
      start = self.get_position()

      self.training = True
      for k in range(self.p.train_episodes):
          state = (start, self.destination)                                                          # Initial state
          while state[0] != self.destination:                                                        # Iterate until agent reaches the goal
              action, valid_move = self.execute()                                                    # Choose & execute action
              new_state = self.get_state()
              reward = self.env.get_reward(new_state[0], self.destination) if valid_move else -1000  # Get action reward
              self.blackboard.update_Q(state, action, reward, new_state)                             # Update Q-values
              state = new_state
          self.env.move_to(self, start)
          self.reward = 0
      self.training = False

    """
    Choose action based on learning
    """
    def execute(self):
      # If in destination, dont move
      if self.get_position() == self.destination:

        # Calculate the amount of steps required to get to the finish
        if self.finish_time == -1 and not self.training:
          self.finish_time = self.model.t
        return (None, True)


      action = self.choose_action(self.get_state())
      action_move = self.actions[action]

      pos = self.get_position()
      new_position = (pos[0] + action_move[0], pos[1] + action_move[1])

      # This move would take the pedestrian out of bounds
      if new_position[0] < 0 or new_position[0] >= self.env.shape[0] or new_position[1] < 0 or new_position[1] >= self.env.shape[1]:
        self.reward -= 1000
        return (action, False)

      # Move to new position if possible and collect reward
      if self.training or self.can_move_into(new_position):
        self.env.move_to(self, new_position)
        self.reward += self.env.get_reward(self.get_position(), self.destination)
        if not self.training:
          self.distance_moved += 1
      else:
        if not self.training:
          self.wait_time += 1

      # Check if agent is on a crosswalk
      if self.env.get_tile(new_position) == pw:
        self.inside_intersection = True
      elif self.env.get_tile(new_position) == sw:
        self.inside_intersection = False

      return (action, True)

    """
    Choose action that is optimum or random based on models epsilon
    """
    def choose_action(self, state):
      if random.uniform(0, 1) < self.blackboard.epsilon:
        return random.choice(list(self.actions.keys()))
      else:
        return max(self.blackboard.Q[state], key=self.blackboard.Q[state].get)

    """
    Get agents position
    """
    def get_position(self):
      return self.env.positions[self]

    """
    Check if agent can move into new position without getting run over
    """
    def can_move_into(self, pos):
      return (self.env.get_tile(pos) != cw or self.inside_intersection) and not self.model.has_car(pos)

    """
    Get next position and if allowed, if not, stay in place
    """
    def get_next_position(self):
      pos = self.get_position()
      if pos == self.destination:
        return pos
      action = self.choose_action(self.get_state())
      action_move = self.actions[action]
      new_position = (pos[0] + action_move[0], pos[1] + action_move[1])

      if self.can_move_into(new_position):
        return new_position
      else:
        return pos

    """
    State of agent is based of position and its destination
    """
    def get_state(self):
      return (self.get_position(), self.destination)

In [7]:
"""
Mapping of tile types and rewards/penalties for training
"""
rewardMap = {
  # Should never go through these (obstacles and buildings)
  ob: -1000,
  ho: -1000,
  de: -1000,

  # Not preferred but possible (streets)
  us: -500,
  ds: -500,
  ls: -500,
  rs: -500,
  ic: -500,

  # Can be used to avoid an obstacle (grass)
  gr: -20,

  # Normal walking spaces (sidewalks, crosswalks and driveways)
  sw: -1,
  dw: -1,
  cw: -1,
  pw: -1
}

class StreetGrid(ap.Grid):
    """
    Setup the street tiles
    """
    def setup(self):
      # Initialize the environment
      self.environment = np.copy(self.p.street)

    """
    Get value of tile
    (agents have different destinations, return reward only if arrived to its desination)
    """
    def get_reward(self, pos, destination):
      if pos == destination:
        return 1000

      return rewardMap[self.get_tile(pos)]

    """
    Get the value of a specified position
    """
    def get_tile(self, pos):
      if 0 <= pos[0] < self.shape[0] and 0 <= pos[1] < self.shape[1]:
        return self.environment[pos]
      else:
        return None

    """
    Change the value of a tile on the grid
    """
    def change_tile(self, pos, val):
      if 0 <= pos[0] < self.shape[0] and 0 <= pos[1] < self.shape[1]:
        self.environment[pos] = val

In [8]:
class Blackboard():

  """
  Black board is used to optimize learning between agents
  All agents with the same destination share the same Q Table
  (This is done by making the state depend on the position, destination and action)
  """
  def __init__(self, model):
    self.actions = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}

    self.alpha = model.p.alpha
    self.gamma = model.p.gamma
    self.epsilon = model.p.epsilon

    self.Q = {}
    for i in range(model.env.shape[0]):
      for j in range(model.env.shape[1]):
        for destination in model.destinations:
          self.Q[((i, j), destination)] = {action: 0 for action in self.actions}

  """
  Update queue value
  """
  def update_Q(self, state, action, reward, new_state):
        max_Q_new_state = max(self.Q[new_state].values())
        self.Q[state][action] = self.Q[state][action] + self.alpha * (reward + self.gamma * max_Q_new_state - self.Q[state][action])

In [9]:
class StreetModel(ap.Model):
    def setup(self):
        self.random.seed = self.p.seed
        self.env = StreetGrid(self, shape=self.p.street.shape, torus=True)
        self.destinations = self.find_tiles(de)
        # Blackboard for training
        self.blackboard = Blackboard(self)

        roads = self.find_tiles(us) + self.find_tiles(ds) + self.find_tiles(ls) + self.find_tiles(rs)
        car_positions = []
        # Spawn cars on the road (according to density)
        for road in roads:
          if self.random.uniform(0, 1) < self.p.car_density:
            car_positions.append(road)

        # Spawn traffic light and cars
        self.traffic_light_agents = ap.AgentList(self, len(self.p.traffic_light_agents), TrafficLightAgent)
        self.car_agents = ap.AgentList(self, len(car_positions), CarAgent)

        # Spawn a pedestrian in each house
        houses = self.find_tiles(ho)
        self.pedestrian_agents = ap.AgentList(self, len(houses), PedestrianAgent)

        # Add all three types of agents
        self.env.add_agents(self.traffic_light_agents, positions=self.p.traffic_light_agents)
        self.env.add_agents(self.car_agents, positions=car_positions)
        self.env.add_agents(self.pedestrian_agents, positions=houses)

        # Train the model (without cars or traffic lights executing to make the training space static)
        print("Training...")
        self.pedestrian_agents.train(self.p.train_episodes)
        self.blackboard.epsilon = 0
        print("Finished training...")

        self.car_agents.choose_direction()
        self.traffic_light_agents.find_exits()

        # Moving agents positions
        self.car_positions = []
        self.pedestrian_positions = []

    """
    Check if there is a car in position
    """
    def has_car(self, pos):
      pos = (pos[0] % self.env.shape[0], pos[1] % self.env.shape[1])

      blocked = False
      agents = list(self.env.grid[pos][0])
      for agent in agents:
        if agent.agentType == "car":
          blocked = True
          break

      return pos in self.car_positions or blocked

    """
    Check if there is a pedestrian in position
    """
    def has_pedestrian(self, pos):
      pos = (pos[0] % self.env.shape[0], pos[1] % self.env.shape[1])

      blocked = False
      agents = list(self.env.grid[pos][0])
      for agent in agents:
        if agent.agentType == "pedestrian":
          blocked = True
          break

      return pos in self.pedestrian_positions or blocked

    """
    Check if tile is crosswalk (abled or enabled)
    """
    def is_crosswalk(self, pos):
      return self.env.get_tile(pos) == pw or self.env.get_tile(pos) == cw

    """
    Check that there isn't any pedestrian moving into the crosswalk
    """
    def crosswalk_has_pedestrians(self, pos):
      crosswalk = []
      # As crosswalk occupate two tiles, check if agent is on a cross walk and which way is the other crosswalk tile
      if self.is_crosswalk(pos):
        crosswalk.append(pos)
        if self.is_crosswalk((pos[0]+1, pos[1])):
          crosswalk.append((pos[0]+1, pos[1]))
        elif self.is_crosswalk((pos[0]-1, pos[1])):
          crosswalk.append((pos[0]-1, pos[1]))
        elif self.is_crosswalk((pos[0], pos[1]+1)):
          crosswalk.append((pos[0], pos[1]+1))
        elif self.is_crosswalk((pos[0], pos[1]-1)):
          crosswalk.append((pos[0], pos[1]-1))

      # Check both crosswalk tiles (if any) for pedestrians
      blocked = False
      for crosswalk_tile in crosswalk:
        agents = list(self.env.grid[crosswalk_tile][0])
        for agent in agents:
          if agent.agentType == "pedestrian":
            blocked = True
          break
        if blocked: break
      return blocked

    """
    Get all positions of a specific type of tile
    """
    def find_tiles(self, tile):
      tiles = []
      for i in range(self.env.shape[0]):
        for j in range(self.env.shape[1]):
          if self.env.get_tile((i, j)) == tile:
            tiles.append((i, j))
      return tiles

    """
    Simulation step
    """
    def step(self):
      # Store the current car positions so that execution order doesnt affect the car movements
      self.car_positions = []
      for car in self.car_agents:
        self.car_positions.append(car.get_position())

      # Store the current pedestrian positions as well as their next position as they have preference in movement
      self.pedestrian_positions = []
      for pedestrian in self.pedestrian_agents:
        self.pedestrian_positions.append(pedestrian.get_position())
        self.pedestrian_positions.append(pedestrian.get_next_position())

      # Execute agent actions
      self.traffic_light_agents.execute()
      self.car_agents.execute()
      self.pedestrian_agents.execute()

      self.env.record_positions()

    """
    Update agent values
    """
    def update(self):
      # Check if all pedestrians have reached their destination
      pedestrian_remaining = False
      for pedestrian in self.pedestrian_agents:
        if pedestrian.get_position() != pedestrian.destination:
          pedestrian_remaining = True
          break

      # Stop if all pedestrians arrived to their destinations
      if not pedestrian_remaining:
        self.stop()
        print("All pedestrians made it to their destination")

    # Report Q-values as well as key statistic findings
    def end(self):
        self.report('q_table', self.blackboard.Q)

        n = len(self.pedestrian_agents)

        avg_trip_time = sum(self.pedestrian_agents.finish_time) / n
        avg_wait_time = sum(self.pedestrian_agents.wait_time) / n
        avg_distance = sum(self.pedestrian_agents.distance_moved) / n

        self.report('average_trip_time', avg_trip_time)
        self.report('average_wait_time', avg_wait_time)
        self.report('average_distance', avg_distance)
        self.report('average_velocity', avg_distance / avg_trip_time)

In [16]:
# Environment representation with a grid
street =  np.array([
    # Big city
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr, sw, ds, us, sw, gr, gr, ho, dw, sw, ds, us, sw, gr, gr, de, dw, sw, ds, us, sw, gr, gr, gr, gr],
    [gr, gr, gr, gr, ob, ds, us, sw, gr, gr, de, dw, sw, ds, us, sw, dw, ho, gr, gr, sw, ds, us, sw, gr, gr, ho, gr, sw, ds, us, sw, dw, ho, gr, gr],
    [ho, gr, ho, gr, sw, ds, us, sw, dw, de, gr, gr, sw, ds, us, sw, gr, gr, ho, gr, ob, ds, us, sw, dw, ho, dw, gr, sw, ds, us, sw, gr, gr, ho, gr],
    [dw, gr, dw, gr, sw, ds, us, sw, gr, gr, gr, gr, sw, ds, us, sw, gr, gr, dw, gr, sw, ds, us, sw, gr, gr, dw, gr, sw, ds, us, sw, gr, gr, dw, gr],
    [sw, sw, sw, sw, sw, cw, cw, sw, ob, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw, sw, cw, cw, sw, ob, sw, ob, sw, sw, cw, cw, sw, ob, sw, sw, sw],
    [ls, ls, ls, ls, cw, ic, ic, cw, ls, ls, ls, ls, cw, ic, ic, cw, ls, ls, ls, ls, cw, ic, ic, cw, ls, ls, ls, ls, cw, ic, ic, cw, ls, ls, ls, ls],
    [rs, rs, rs, rs, cw, ic, ic, cw, rs, rs, rs, rs, cw, ic, ic, cw, rs, rs, rs, rs, cw, ic, ic, cw, rs, rs, rs, rs, cw, ic, ic, cw, rs, rs, rs, rs],
    [sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw],
    [gr, dw, gr, gr, sw, ds, us, sw, gr, gr, gr, gr, sw, ds, us, sw, gr, gr, ho, dw, sw, ds, us, sw, gr, gr, gr, gr, sw, ds, us, sw, gr, dw, gr, gr],
    [gr, ho, gr, gr, sw, ds, us, sw, gr, gr, ho, dw, sw, ds, us, ob, dw, de, gr, gr, sw, ds, us, sw, dw, ho, ho, dw, ob, ds, us, sw, gr, de, gr, gr],
    [gr, gr, de, dw, sw, ds, us, sw, dw, ho, gr, gr, ob, ds, us, sw, gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr, sw, ds, us, sw, ob, gr, gr, gr],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, ho, dw, sw, ds, us, sw, gr, gr, gr, gr, ob, ds, us, sw, dw, ho, ho, dw, sw, ds, us, sw, dw, ho, gr, gr]
    # Small city
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    # [sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw],
    # [ls, ls, ls, ls, cw, ic, ic, cw, ls, ls, ls, ls],
    # [rs, rs, rs, rs, cw, ic, ic, cw, rs, rs, rs, rs],
    # [sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw],
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    # [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr]
])

parameters = {
    'street': street,
    # Big city
    'traffic_light_agents': [(5, 5), (5, 13), (5, 21), (5, 29)],  # It is assumed the agent spawns in an 'ic' (intersection) location
    # Small city
    # 'traffic_light_agents': [(5, 5)],
    'traffic_light_step_duration': 12,
    'traffic_light_step_cooldown': 4,
    'car_density': 0.3,
    'steps': 128,
    'seed': 2,
    'alpha': 0.9,
    'gamma': 1.0,
    'epsilon': 0.3,
    'train_episodes': 200
}

In [19]:
"""
Function to create an animation at each step of the simulation
"""
def animation_plot(model, ax):
    N, M = model.p.street.shape
    grid = np.copy(model.p.street)

    # Iterate over all traffic light agents
    for traffic_light_agent in model.traffic_light_agents:
        traffic_light_state = model.env.positions[traffic_light_agent]
        # Make it a street tile for display
        grid[traffic_light_state] = us

        # Paint parallel crosswalks to traffic light
        for tile in traffic_light_agent.cross_walks[traffic_light_agent.intersection_position]:
          grid[tile] = pw
        for tile in traffic_light_agent.cross_walks[traffic_light_agent.get_opposite_direction(traffic_light_agent.intersection_position)]:
          grid[tile] = pw

    # Find the position of all red lights (left over intersection tiles)
    red_lights = []
    for i in range(N):
      for j in range(M):
        if grid[(i, j)] == ic:
          red_lights.append((i, j))
          grid[(i, j)] = us

    # Place the cars in the display grid
    for car_agent in model.car_agents:
      car_state = model.env.positions[car_agent]
      grid[car_state] = car

    # Helper dictionaries and arrays for drawing colors and direction icons
    directionArrows = ["⬆︎", "⬅︎", "⬇︎", "➡︎"]
    color_dict = {gr:'#2c7d29', cw: '#cccccc', ho: "#b34b29", ob: "#806332", de: "#435ee6", dw: "#81848a", ic: '#ff0000', us: '#777777', ls: '#777777', ds:'#777777', rs:'#777777', car: "#1e152a", traffic_light:'#00ff00', traffic_light_cooldown: '#ffff00', sw: '#a1a7b5', pw: '#999999'}
    directionDict = {us: 0, ls: 1, ds: 2, rs: 3}

    ap.gridplot(grid, ax=ax, color_dict=color_dict, convert=True)

    # Paint arrows as text
    for i in range(N):
      for j in range(M):
        tile = model.env.get_tile((i, j))
        if tile in directionDict and (i,j) not in model.env.positions.values():
          ax.text(j, i, directionArrows[directionDict[tile]], color = "#b0b0b0")

    # Draw traffic lights as dots (Green or yellow)
    for traffic_light_agent in model.traffic_light_agents:
      state = model.env.positions[traffic_light_agent]
      if traffic_light_agent.cooldown:
        ax.text(state[1]-0.25, state[0]+0.25, "•", color="#ffff00", size=20)
      else:
        ax.text(state[1]-0.25, state[0]+0.25, "•", color="#00ff00", size=20)

    # Draw traffic lights as dots (Red)
    for red_light in red_lights:
      ax.text(red_light[1]-0.25, red_light[0]+0.25, "•", color="#ff0000", size=20)

    # Draw pedestrians as dots (White) with their offsets
    for pedestrian in model.pedestrian_agents:
      pos = pedestrian.get_position()
      ax.text(pos[1]-0.2+pedestrian.display_offset[1], pos[0]+0.2+pedestrian.display_offset[0], "•", color="white", size=15)
      i += 1

model = StreetModel(parameters)
fig = plt.figure(figsize=(14,7))
ax = fig.add_subplot(111)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())

Training...
Finished training...
All pedestrians made it to their destination


In [21]:
# Run the simulation without animations and show results
model = StreetModel(parameters)
stats = model.run()
results = stats.arrange_reporters()

print("Average trip time: " + str(round(results["average_trip_time"][0], 2)) + " steps")
print("Average wait time: " + str(round(results["average_wait_time"][0], 2)) + " steps")
print("Average distance travelled: " + str(round(results["average_distance"][0], 2)) + " tiles")
print("Average velocity: " + str(round(results["average_velocity"][0], 2)) + " tiles per step")

Training...
Finished training...
Completed: 70 stepsAll pedestrians made it to their destination
Completed: 71 steps
Run time: 0:00:04.596352
Simulation finished
Average trip time: 36.0 steps
Average wait time: 14.89 steps
Average distance travelled: 23.95 tiles
Average velocity: 0.67 tiles per step


# API

## /start_simulation [POST]

## /simulation_stats [GET]

Method to get simulation meta stadisditcs

```js
{
    'traffic_light_step_duration': int,
    'traffic_light_step_cooldown': int,
    'car_density': double,
    'steps': int,
    'alpha': double,
    'gamma': double,
    'epsilon': double,
    'train_episodes': 200,
    'car_id_list': [int],
    'pedestrian_id_list': [int],
    'traffic_light_id_list': [int],
}
```

## /mobile_agent_state/{step_number} [GET]

Method to get the location of mobile agents in a specific step

```js
[
  {
    'id': int,
    'x': int,
    'y': int,
    'type': string
  },
  ...
]
```

## /static_agent_state/{step_number} [GET]

```js
[
  {
    'id': int,
    'x': int,
    'y': int,
    'status': bool (True = Green, False = Yellow)
  },
  ...
]
```



In [22]:
"""
Flask REST API
"""

from flask import Flask, request, jsonify
import pandas as pd

# Gather the simulation variables which include the simulation step information
df = stats.arrange_variables()
app = Flask(__name__)

"""
GET route to show different simulation statistics
"""
@app.route('/simulation_stats', methods=['GET'])
def get_simulation_stats():
    agent_ids = df.groupby('obj_type')['obj_id'].unique().apply(list).to_dict()
    return jsonify({**parameters, **agent_ids})

"""
GET route that allows a user to get the state of all mobile agents (cars and pedestrians) at a specific time step
"""
@app.route('/mobile_agent_state/<int:step_number>', methods=['GET'])
def get_mobile_agent_state(step_number):
    # Filter the dataframe to only include pedestrians and cars for the specified time step
    filtered_df = df[(df['t'] == step_number) & df['obj_type'].isin(['PedestrianAgent', 'CarAgent'])]
    result = [{'id': row['obj_id'], 'x': row['p0'], 'y': row['p1'], 'type':row['obj_type']} for _, row in filtered_df.iterrows()]
    print(result)
    return jsonify(result) if result else jsonify({'error': 'No data for the given step'})

"""
GET route that allows a user to get the state of all static agents (traffic lights) at a specific time step
"""
@app.route('/static_agent_state/<int:step_number>', methods=['GET'])
def get_static_agent_state(step_number):
    # Filter the dataframe to only include traffic lights for the specified time step
    filtered_df = df[(df['t'] == step_number) & (df['obj_type'] == 'TrafficLightAgent')]
    result = [{'id': row['obj_id'], 'x': row['p0'], 'y': row['p1'], 'status': row['status']} for _, row in filtered_df.iterrows()]
    return jsonify(result) if result else jsonify({'error': 'No data for the given step'})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
"""
API test calls
"""

import requests

BASE_URL = "http://127.0.0.1:5000"

def test_simulation_stats():
    response = requests.get(f"{BASE_URL}/simulation_stats")
    print("/simulation_stats:", response.status_code, response.json())

def test_mobile_agent_state(step):
    response = requests.get(f"{BASE_URL}/mobile_agent_state/{step}")
    print(f"/mobile_agent_state/{step}:", response.status_code, response.json())

def test_static_agent_state(step):
    response = requests.get(f"{BASE_URL}/static_agent_state/{step}")
    print(f"/static_agent_state/{step}:", response.status_code, response.json())

if __name__ == "__main__":
    test_simulation_stats()
    test_mobile_agent_state(1)
    test_static_agent_state(1)
